# 📊 Análisis Automatizado Avanzado de Datos con IA
## Versión 2: Análisis Completo + ML + PCA + Clustering + Claude

**Análisis exhaustivo automático de cualquier CSV**

### Requisitos: Mínimo 2,000 registros y 10 columnas

---
## ⚙️ CONFIGURACIÓN - MODIFICA SOLO ESTA CELDA

In [ ]:
# ============================================================================
# 🔧 CONFIGURACIÓN
# ============================================================================

CSV_PATH = "data/tu_archivo.csv"  # Ruta al CSV

DATASET_DESCRIPTION = """
Describe tu dataset aquí (2-3 oraciones).
Ejemplo: Dataset de ventas con información de productos y clientes.
"""

DATASET_NAME = "Mi Dataset"
TARGET_VARIABLE = None  # Variable objetivo (None = auto-detectar)
PROBLEM_TYPE = None  # 'clasificacion', 'regresion', o None (auto)
CSV_SEPARATOR = ","
N_CLUSTERS = None  # Número de clusters (None = automático)

print(f"Configuración: {CSV_PATH}")

---
## 1- Importar Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import shapiro, normaltest, f_oneway, kruskal
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import *
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import warnings, os
from dotenv import load_dotenv

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
load_dotenv()
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
os.makedirs('outputs', exist_ok=True)
os.makedirs('reports', exist_ok=True)
print('Librerías cargadas')

---
## 2- Cargar y Validar Datos

In [ ]:
df = pd.read_csv(CSV_PATH, sep=CSV_SEPARATOR)
df = df.dropna(axis=1, how='all')
df.columns = df.columns.str.strip()
print(f'Cargado: {df.shape[0]:,} x {df.shape[1]}')
print(f"{'✅' if df.shape[0]>=2000 else '❌'} Registros: {df.shape[0]:,}")
print(f"{'✅' if df.shape[1]>=10 else '❌'} Columnas: {df.shape[1]}")

In [ ]:
# Identificar tipos de variables automáticamente
columnas_numericas = df.select_dtypes(include=[np.number]).columns.tolist()
columnas_objeto = df.select_dtypes(include=['object', 'category']).columns.tolist()
columnas_categoricas = columnas_objeto.copy()
for col in columnas_numericas:
    if df[col].nunique() <= 15:
        columnas_categoricas.append(col)
columnas_continuas = [c for c in columnas_numericas if c not in columnas_categoricas]

# Detectar variable objetivo
if TARGET_VARIABLE and TARGET_VARIABLE in df.columns:
    variable_objetivo = TARGET_VARIABLE
elif columnas_categoricas:
    variable_objetivo = columnas_categoricas[-1]
else:
    variable_objetivo = columnas_numericas[-1] if columnas_numericas else None

# Tipo de problema
if PROBLEM_TYPE:
    tipo_problema = PROBLEM_TYPE
elif variable_objetivo and (variable_objetivo in columnas_categoricas or df[variable_objetivo].nunique() <= 15):
    tipo_problema = 'clasificacion'
else:
    tipo_problema = 'regresion'

print(f'\nContinuas: {len(columnas_continuas)} | Categóricas: {len(columnas_categoricas)}')
print(f'Objetivo: {variable_objetivo} | Tipo: {tipo_problema}')

In [ ]:
df.head(10)

---
## 3- Valores Faltantes

In [ ]:
valores_faltantes = df.isnull().sum()
porcentaje_faltantes = (df.isnull().sum() / len(df)) * 100
print(f'Total faltantes: {df.isnull().sum().sum():,} ({porcentaje_faltantes.mean():.2f}%)')

fig, ax = plt.subplots(figsize=(14, 5))
sns.heatmap(df.isnull(), cbar=True, yticklabels=False, cmap='viridis')
plt.title(f'Valores Faltantes - {DATASET_NAME}', fontweight='bold')
plt.tight_layout()
plt.savefig('outputs/01_heatmap_faltantes.png', dpi=150)
plt.show()

---
## 4- Estadísticas Descriptivas

In [ ]:
if columnas_continuas:
    estadisticas = df[columnas_continuas].describe().T
    estadisticas['mediana'] = df[columnas_continuas].median()
    estadisticas['asimetria'] = df[columnas_continuas].skew()
    estadisticas['curtosis'] = df[columnas_continuas].kurtosis()
    display(estadisticas.round(4))
else:
    estadisticas = pd.DataFrame()

---
## 5- Tests de Normalidad

In [ ]:
resultados_normalidad = []
for col in columnas_continuas:
    muestra = df[col].dropna().sample(min(5000, len(df)), random_state=42)
    try:
        _, p_shap = shapiro(muestra)
        _, p_dag = normaltest(muestra)
        normal = 'Sí' if p_shap > 0.05 and p_dag > 0.05 else 'No'
    except:
        p_shap, p_dag, normal = None, None, 'Error'
    resultados_normalidad.append({'Variable': col, 'Shapiro (p)': p_shap, "D'Agostino (p)": p_dag, 'Normal': normal})
df_normalidad = pd.DataFrame(resultados_normalidad)
display(df_normalidad)

---
## 6- Visualizaciones

In [ ]:
# Histogramas
if columnas_continuas:
    n = min(len(columnas_continuas), 12)
    nc = 3
    nr = (n + nc - 1) // nc
    fig, axes = plt.subplots(nr, nc, figsize=(15, 4*nr))
    axes = axes.flatten() if nr > 1 else [axes]
    for i, col in enumerate(columnas_continuas[:n]):
        sns.histplot(df[col].dropna(), bins=40, kde=True, ax=axes[i])
        axes[i].axvline(df[col].mean(), color='red', linestyle='--')
        axes[i].set_title(col, fontweight='bold')
    for j in range(i+1, len(axes)): axes[j].set_visible(False)
    plt.tight_layout()
    plt.savefig('outputs/02_histogramas.png', dpi=150)
    plt.show()

In [ ]:
# Boxplots
if columnas_continuas:
    fig, axes = plt.subplots(nr, nc, figsize=(15, 4*nr))
    axes = axes.flatten() if nr > 1 else [axes]
    for i, col in enumerate(columnas_continuas[:n]):
        bp = axes[i].boxplot(df[col].dropna(), patch_artist=True)
        bp['boxes'][0].set_facecolor('lightblue')
        Q1, Q3 = df[col].quantile(0.25), df[col].quantile(0.75)
        outliers = ((df[col] < Q1 - 1.5*(Q3-Q1)) | (df[col] > Q3 + 1.5*(Q3-Q1))).sum()
        axes[i].set_title(f'{col} ({outliers} outliers)', fontweight='bold')
    for j in range(i+1, len(axes)): axes[j].set_visible(False)
    plt.tight_layout()
    plt.savefig('outputs/03_boxplots.png', dpi=150)
    plt.show()

---
## 7- Outliers

In [ ]:
resultados_outliers = []
total_outliers = 0
for col in columnas_continuas:
    Q1, Q3 = df[col].quantile(0.25), df[col].quantile(0.75)
    IQR = Q3 - Q1
    out = ((df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)).sum()
    total_outliers += out
    resultados_outliers.append({'Variable': col, 'Outliers': out, '%': out/len(df)*100})
df_outliers = pd.DataFrame(resultados_outliers).sort_values('Outliers', ascending=False)
print(f'Total outliers: {total_outliers:,}')
display(df_outliers)

---
## 8- Correlaciones

In [ ]:
if len(columnas_numericas) > 1:
    corr_pearson = df[columnas_numericas].corr()
    fig, ax = plt.subplots(figsize=(12, 10))
    mask = np.triu(np.ones_like(corr_pearson, dtype=bool))
    sns.heatmap(corr_pearson, mask=mask, annot=len(columnas_numericas)<=12, fmt='.2f', cmap='RdBu_r', center=0)
    plt.title('Matriz de Correlaciones', fontweight='bold')
    plt.tight_layout()
    plt.savefig('outputs/04_correlaciones.png', dpi=150)
    plt.show()
    
    # Top correlaciones
    pares = []
    for i in range(len(corr_pearson.columns)):
        for j in range(i+1, len(corr_pearson.columns)):
            pares.append((corr_pearson.columns[i], corr_pearson.columns[j], corr_pearson.iloc[i,j]))
    pares.sort(key=lambda x: abs(x[2]), reverse=True)
    df_top_corr = pd.DataFrame(pares[:10], columns=['Var1', 'Var2', 'Corr'])
    display(df_top_corr)
else:
    corr_pearson = pd.DataFrame()
    df_top_corr = pd.DataFrame()

---
## 9- Tests Estadísticos

In [ ]:
resultados_tests = []
if variable_objetivo and tipo_problema == 'clasificacion':
    for col in columnas_continuas:
        grupos = [df[df[variable_objetivo]==g][col].dropna() for g in df[variable_objetivo].unique()]
        grupos = [g for g in grupos if len(g) > 0]
        if len(grupos) >= 2:
            try:
                _, p_anova = f_oneway(*grupos)
                _, p_krus = kruskal(*grupos)
                sig = 'Sí' if p_krus < 0.05 else 'No'
            except:
                p_anova, p_krus, sig = None, None, 'Error'
            resultados_tests.append({'Variable': col, 'ANOVA (p)': p_anova, 'Kruskal (p)': p_krus, 'Sig': sig})
df_tests = pd.DataFrame(resultados_tests)
if len(df_tests) > 0:
    vars_sig = df_tests[df_tests['Sig']=='Sí']['Variable'].tolist()
    print(f'Variables significativas: {len(vars_sig)} de {len(columnas_continuas)}')
    display(df_tests)
else:
    vars_sig = []

---
## 10- PCA

In [ ]:
if len(columnas_continuas) >= 2:
    df_pca = df[columnas_continuas].dropna()
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_pca)
    pca = PCA()
    X_pca = pca.fit_transform(X_scaled)
    var_exp = pca.explained_variance_ratio_
    var_acum = np.cumsum(var_exp)
    n_80 = np.argmax(var_acum >= 0.80) + 1
    n_95 = np.argmax(var_acum >= 0.95) + 1
    print(f'PC para 80%: {n_80} | PC para 95%: {n_95}')
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    axes[0].bar(range(1, len(var_exp)+1), var_exp*100)
    axes[0].set_title('Scree Plot', fontweight='bold')
    axes[1].plot(range(1, len(var_acum)+1), var_acum*100, 'bo-')
    axes[1].axhline(80, color='r', linestyle='--')
    axes[1].set_title('Varianza Acumulada', fontweight='bold')
    plt.tight_layout()
    plt.savefig('outputs/05_pca.png', dpi=150)
    plt.show()
else:
    X_pca, var_exp, var_acum, n_80, n_95 = None, [], [], 0, 0

---
## 11- Clustering

In [ ]:
if X_pca is not None and len(X_scaled) > 100:
    sils = []
    for k in range(2, min(11, len(X_scaled)//10)):
        km = KMeans(n_clusters=k, random_state=42, n_init=10)
        km.fit(X_scaled)
        sils.append(silhouette_score(X_scaled, km.labels_))
    mejor_k = range(2, len(sils)+2)[np.argmax(sils)]
    n_clusters = N_CLUSTERS if N_CLUSTERS else min(mejor_k, 4)
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(X_scaled)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='viridis', alpha=0.6)
    cent_pca = pca.transform(kmeans.cluster_centers_)
    ax.scatter(cent_pca[:, 0], cent_pca[:, 1], c='red', marker='X', s=200)
    ax.set_title(f'Clusters K-Means (K={n_clusters})', fontweight='bold')
    plt.savefig('outputs/06_clustering.png', dpi=150)
    plt.show()
    
    df_temp = df.loc[df[columnas_continuas].dropna().index].copy()
    df_temp['cluster'] = clusters
    perfil_clusters = df_temp.groupby('cluster')[columnas_continuas].mean()
    display(perfil_clusters.round(4))
else:
    n_clusters = 0
    perfil_clusters = pd.DataFrame()

---
## 12- Machine Learning

In [ ]:
df_resultados_ml = pd.DataFrame()
mejor_modelo = None
df_importancia = pd.DataFrame()

if variable_objetivo and variable_objetivo in df.columns:
    features = [c for c in columnas_continuas if c != variable_objetivo]
    if not features:
        features = [c for c in columnas_numericas if c != variable_objetivo]
    
    df_ml = df[features + [variable_objetivo]].dropna()
    X = df_ml[features]
    y = df_ml[variable_objetivo]
    if y.dtype == 'object':
        y = LabelEncoder().fit_transform(y)
    
    if tipo_problema == 'clasificacion':
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    scaler_ml = StandardScaler()
    X_train_s = scaler_ml.fit_transform(X_train)
    X_test_s = scaler_ml.transform(X_test)
    
    print(f'Train: {len(X_train):,} | Test: {len(X_test):,}')

In [ ]:
# Entrenar modelos
if variable_objetivo and tipo_problema == 'clasificacion':
    modelos = {
        'Logistic': LogisticRegression(max_iter=1000, random_state=42),
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
        'KNN': KNeighborsClassifier(n_neighbors=5),
        'Decision Tree': DecisionTreeClassifier(random_state=42)
    }
    res = []
    mejor_acc = 0
    for nom, mod in modelos.items():
        mod.fit(X_train_s, y_train)
        y_pred = mod.predict(X_test_s)
        cv = cross_val_score(mod, X_train_s, y_train, cv=5)
        acc = accuracy_score(y_test, y_pred)
        if acc > mejor_acc:
            mejor_acc = acc
            mejor_modelo = (nom, mod)
        res.append({'Modelo': nom, 'Accuracy': acc, 'Precision': precision_score(y_test, y_pred, average='weighted', zero_division=0),
                   'Recall': recall_score(y_test, y_pred, average='weighted', zero_division=0),
                   'F1': f1_score(y_test, y_pred, average='weighted', zero_division=0), 'CV': cv.mean()})
    df_resultados_ml = pd.DataFrame(res).sort_values('Accuracy', ascending=False)
    display(df_resultados_ml)

elif variable_objetivo and tipo_problema == 'regresion':
    modelos = {
        'Linear': LinearRegression(),
        'Ridge': Ridge(random_state=42),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
    }
    res = []
    mejor_r2 = -999
    for nom, mod in modelos.items():
        mod.fit(X_train_s, y_train)
        y_pred = mod.predict(X_test_s)
        r2 = r2_score(y_test, y_pred)
        if r2 > mejor_r2:
            mejor_r2 = r2
            mejor_modelo = (nom, mod)
        res.append({'Modelo': nom, 'R²': r2, 'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)), 'MAE': mean_absolute_error(y_test, y_pred)})
    df_resultados_ml = pd.DataFrame(res).sort_values('R²', ascending=False)
    display(df_resultados_ml)

In [ ]:
# Feature Importance
if mejor_modelo and hasattr(mejor_modelo[1], 'feature_importances_'):
    imp = mejor_modelo[1].feature_importances_
    idx = np.argsort(imp)[::-1]
    df_importancia = pd.DataFrame({'Variable': [features[i] for i in idx], 'Importancia': [imp[i] for i in idx]})
    
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.barh(df_importancia['Variable'], df_importancia['Importancia'], color=plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(df_importancia))))
    ax.invert_yaxis()
    ax.set_title(f'Feature Importance - {mejor_modelo[0]}', fontweight='bold')
    plt.tight_layout()
    plt.savefig('outputs/07_feature_importance.png', dpi=150)
    plt.show()
    display(df_importancia)

In [ ]:
# Matriz de confusión
if mejor_modelo and tipo_problema == 'clasificacion':
    y_pred = mejor_modelo[1].predict(X_test_s)
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
    ax.set_title(f'Matriz de Confusión - {mejor_modelo[0]}', fontweight='bold')
    plt.tight_layout()
    plt.savefig('outputs/08_confusion_matrix.png', dpi=150)
    plt.show()

---
## 13- Generar Resumen para Claude

In [ ]:
resumen = f'''
RESUMEN DEL ANÁLISIS - {DATASET_NAME}
{'='*60}

DESCRIPCIÓN: {DATASET_DESCRIPTION}

1. DATASET: {df.shape[0]:,} registros x {df.shape[1]} columnas
   - Continuas: {len(columnas_continuas)} | Categóricas: {len(columnas_categoricas)}
   - Objetivo: {variable_objetivo} | Tipo: {tipo_problema}
   - Duplicados: {df.duplicated().sum():,}

2. CALIDAD: Faltantes: {df.isnull().sum().sum():,} ({porcentaje_faltantes.mean():.2f}%)
   Outliers: {total_outliers:,}

3. ESTADÍSTICAS:
{estadisticas[["mean", "std", "min", "max"]].to_string() if len(estadisticas) > 0 else "N/A"}

4. NORMALIDAD:
{df_normalidad.to_string() if len(df_normalidad) > 0 else "N/A"}

5. TOP CORRELACIONES:
{df_top_corr.to_string() if len(df_top_corr) > 0 else "N/A"}

6. TESTS: Variables significativas: {len(vars_sig) if "vars_sig" in dir() else 0}

7. PCA: Componentes 80%: {n_80} | 95%: {n_95}

8. CLUSTERING: {n_clusters} clusters

9. ML RESULTADOS:
{df_resultados_ml.to_string() if len(df_resultados_ml) > 0 else "N/A"}
   Mejor: {mejor_modelo[0] if mejor_modelo else "N/A"}

10. FEATURE IMPORTANCE:
{df_importancia.to_string() if len(df_importancia) > 0 else "N/A"}
'''

with open('reports/resumen_analisis_v2.txt', 'w', encoding='utf-8') as f:
    f.write(resumen)
print('Guardado: reports/resumen_analisis_v2.txt')
print(resumen[:2000])

---
# 14- Generar Insights con Claude

In [ ]:
from anthropic import Anthropic
api_key = os.getenv('ANTHROPIC_API_KEY')
if not api_key:
    print('Configura ANTHROPIC_API_KEY en .env')
else:
    print('API Key encontrada')

In [ ]:
if api_key:
    print('Generando insights...')
    client = Anthropic(api_key=api_key)
    
    prompt = f'''
Eres experto en análisis de datos y ML. Analiza este resumen y genera un REPORTE EJECUTIVO con:

1. RESUMEN EJECUTIVO (3 párrafos)
2. CALIDAD DE DATOS
3. HALLAZGOS CLAVE (5)
4. ANÁLISIS DE ML
5. ANÁLISIS PCA/CLUSTERING
6. RECOMENDACIONES (5)
7. LIMITACIONES
8. CONCLUSIONES Y PRÓXIMOS PASOS

CONTEXTO: {DATASET_DESCRIPTION}

DATOS:
{resumen}

Responde en español, profesionalmente, usando datos específicos.
'''
    
    try:
        response = client.messages.create(
            model='claude-sonnet-4-20250514',
            max_tokens=6000,
            messages=[{'role': 'user', 'content': prompt}]
        )
        insights = response.content[0].text
        
        with open('reports/insights_claude_v2.txt', 'w', encoding='utf-8') as f:
            f.write(f'REPORTE - {DATASET_NAME}\n{"="*60}\n\n{insights}')
        
        print('='*60)
        print('REPORTE GENERADO')
        print('='*60)
        print(insights)
        print('\nGuardado: reports/insights_claude_v2.txt')
    except Exception as e:
        print(f'Error: {e}')

---
##Resumen Final

In [ ]:
print('='*60)
print('ANÁLISIS COMPLETADO')
print('='*60)
print(f'''
{DATASET_NAME}
{df.shape[0]:,} x {df.shape[1]}

Gráficas generadas en outputs/
📄 Reportes en reports/

Métricas:
   • Outliers: {total_outliers:,}
   • Clusters: {n_clusters}
   • Mejor modelo: {mejor_modelo[0] if mejor_modelo else "N/A"}
   • Top variable: {df_importancia.iloc[0]["Variable"] if len(df_importancia) > 0 else "N/A"}
''')
print('\n¡Completado!')